In [1]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataRep = '../data/'
datasetRep = '../data/dataset/'
scriptRep = '../script/'

# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, scriptRep)
import progressBar
from datasetCreation import createDatasetSecondNetwork

unicodeData = pd.read_csv(dataRep+'unicode_translation.csv')
trainData = pd.read_csv(dataRep+'train.csv')
sampleSubData = pd.read_csv(dataRep+'sample_submission.csv')

In [ ]:
createDatasetSecondNetwork(dataRep+'train.csv', datasetRep + 'test.npz')

  0%|          | 0/3881 [00:00<?, ?it/s]

Convert train image


 68%|██████▊   | 2622/3881 [1:07:44<1:57:17,  5.59s/it]

In [ ]:
raw = np.load('../data/dataset/test.npz')

nDisp = 6
imageId = np.random.randint(raw['image'].shape[0], size=(nDisp, nDisp))
print(raw['image'].shape, raw['character'].shape, np.unique(raw['character']).shape, raw['characterMap'].shape)
print(np.unique(raw['character']))

plt.figure(figsize=(15,15))
for i in range(nDisp):
    for j in range(nDisp):
        plt.subplot(nDisp, nDisp, i*nDisp+j+1)
        plt.imshow(raw['image'][imageId[i,j],:,:], cmap='gist_gray')
plt.show()

In [ ]:
raw['image'].nbytes/1024/1024

In [ ]:
raw['character'].nbytes/1024/1024

In [ ]:
raw['characterMap'].nbytes/1024/1024

In [ ]:
raw['characterClass'].nbytes/1024/1024